# Tutorial 4 - Certainty factors

This tutorial uses the fuzzy inference system developed in Tutorial 1.

In [5]:
import os
import warnings

os.chdir('/workspaces/fuzzy-expert')
warnings.filterwarnings("ignore")

## Fuzzy Variables

In [6]:
import matplotlib.pyplot as plt
import numpy as np

from fuzzy_expert.variable import FuzzyVariable
from fuzzy_expert.rule import FuzzyRule
from fuzzy_expert.inference import DecompositionalInference

variables = {
    "score": FuzzyVariable(
        universe_range=(150, 200),
        terms={
            "High": [(175, 0), (180, 0.2), (185, 0.7), (190, 1)],
            "Low": [(155, 1), (160, 0.8), (165, 0.5), (170, 0.2), (175, 0)],
        },
    ),
    "ratio": FuzzyVariable(
        universe_range=(0.1, 1),
        terms={
            "Goodr": [(0.3, 1), (0.4, 0.7), (0.41, 0.3), (0.42, 0)],
            "Badr": [(0.44, 0), (0.45, 0.3), (0.5, 0.7), (0.7, 1)],
        },
    ),
    #
    "credit": FuzzyVariable(
        universe_range=(0, 10),
        terms={
            "Goodc": [(2, 1), (3, 0.7), (4, 0.3), (5, 0)],
            "Badc": [(5, 0), (6, 0.3), (7, 0.7), (8, 1)],
        },
    ),
    #
    "decision": FuzzyVariable(
        universe_range=(0, 10),
        terms={
            "Approve": [(5, 0), (6, 0.3), (7, 0.7), (8, 1)],
            "Reject": [(2, 1), (3, 0.7), (4, 0.3), (5, 0)],
        },
    ),
}

## Fuzzy rules with certainty factors

It is possible to assign a certainty factor (`cf`) to each rule. If this value is not specified, it has assumed to be equal to 1.0. In addition, the `threshold_cf` is the minimum certainty factor required to consider the rule fired; this is, rules with a computed certainty factor below the threshold are not considering for computing the output of the system. The first rule has a certainty factor of `0.9`, while the second rule has a certainty factor of `1.0` (by default).

In [7]:
rules = [
    FuzzyRule(
        cf=0.9,
        premise=[
            ("score", "High"),
            ("AND", "ratio", "Goodr"),
            ("AND", "credit", "Goodc"),
        ],
        consequence=[("decision", "Approve")],
    ),
    FuzzyRule(
        premise=[
            ("score", "Low"),
            ("AND", "ratio", "Badr"),
            ("OR", "credit", "Badc"),
        ],
        consequence=[("decision", "Reject")],
    )
]

## Facts with certainty factors

In addition, also it is possible to assign certainty factors to the facts. When a certainty factor not is specified by the user, it has a default value or 1.0. In the following code, the variables `score`, `ratio`, and `credit` have certainty factors of `0.9`, `1.0`, and `0.95` respectively. The conclusion is `decision=8.01` with a certainty factor of `0.95`.

In [8]:
from fuzzy_expert.inference import DecompositionalInference

model = DecompositionalInference(
    and_operator="min",
    or_operator="max",
    implication_operator="Rc",
    composition_operator="max-min",
    production_link="max",
    defuzzification_operator="cog",
)

model(
    variables=variables,
    rules=rules,
    score=(190, 0.9),
    ratio=(0.39, 1.0),
    credit=(1.5, 0.95),
)

({'decision': 8.010492631084489}, 0.95)